In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Conv1DTranspose
from tensorflow.keras.constraints import max_norm
import datos_con_clicks as dc
from scipy.io import wavfile


In [31]:
#Load noisy and clean data
clean, noisy = dc.load_data(44000, 60, 20, 0.0034)

In [4]:
#Normalize and build training and test sets
n = len(clean)
train = clean[0:int(0.9*n)]
test = clean[int(0.9*n):n]
train_noisy = noisy[0:int(0.9*n)]
test_noisy = noisy[int(0.9*n):n]
train, train_noisy, test, test_noisy, m, M = dc.normalizar(train, train_noisy, test, test_noisy)

In [ ]:
#In case of only wanting to try pretrained model:
model = tf.keras.models.load_model('working.h5')

In [5]:
input_shape = (train.shape[1], 1)
batch_size = 4
no_epochs = 200
verbosity = 1
max_norm_value = 10

In [ ]:
#Define the model
model = Sequential()
model.add(Conv1D(128, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='leaky_relu', kernel_initializer='he_uniform', input_shape=input_shape))
model.add(Conv1D(64, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='leaky_relu', kernel_initializer='he_uniform'))
model.add(Conv1D(32, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='leaky_relu', kernel_initializer='he_uniform'))
model.add(Conv1DTranspose(128, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='leaky_relu', kernel_initializer='he_uniform'))
model.add(Conv1DTranspose(64, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='leaky_relu', kernel_initializer='he_uniform'))
model.add(Conv1DTranspose(32, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='leaky_relu', kernel_initializer='he_uniform'))
model.add(Conv1D(1, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='sigmoid', padding='same'))

model.summary()

In [ ]:
#Train
model.compile(optimizer='adam', loss='mse')
history = model.fit(train_noisy, train,
                epochs=no_epochs,
                batch_size=batch_size,
                verbose = verbosity,
                validation_data=(test_noisy, test))

In [ ]:
# Loss chart
fig, ax = plt.subplots(figsize=(12,5), dpi=100)
plt.title(label='Pérdida por época', loc='center')

ax.plot(history.history['loss'], label='Entrenamiento', color='black')
ax.plot(history.history['val_loss'], label='Testeo', color='red')
ax.set(xlabel='Epoch', ylabel='Loss')
ax.set_yscale('log')
plt.legend()

plt.show()

In [ ]:
#Predict with the model
testDenoised = model.predict(test_noisy).reshape(test.shape[0], test.shape[1])

#Show some normalized signals
fig, axs = plt.subplots(2, 2, sharey=False, tight_layout=True, figsize=(12,6), facecolor='white')
axs[0,1].plot(testDenoised[10], label='Depurada')
axs[0,1].plot(test_noisy[10], label='Ruidosa')
axs[0,1].plot(test[10], label='Original')
axs[0,1].legend(loc="upper left")
axs[0,1].set(title="Señal 10 (normalizada)")
axs[0,1].grid()
axs[0,1].set_xlabel('Muestras')
axs[0,1].set_ylabel('Magnitud')
axs[0,0].plot(testDenoised[1], label='Depurada')
axs[0,0].plot(test_noisy[1], label='Ruidosa')
axs[0,0].plot(test[1], label= 'Original')
axs[0,0].legend(loc="upper left")
axs[0,0].grid()
axs[0,0].set_xlabel('Muestras')
axs[0,0].set_ylabel('Magnitud')
axs[0,0].set(title="Señal 1 ruidosa (normalizada)")
axs[1,1].plot(testDenoised[40], label='Depurada')
axs[1,1].plot(test_noisy[40], label='Ruidosa')
axs[1,1].plot(test[40], label='Original')
axs[1,1].legend(loc="lower left")
axs[1,1].grid()
axs[1,1].set_xlabel('Muestras')
axs[1,1].set_ylabel('Magnitud')
axs[1,1].set(title="Señal 50 luego de pasar por la red (normalizada)")
axs[1,0].plot(testDenoised[3], label='Depurada')
axs[1,0].plot(test_noisy[3], label='Ruidosa')
axs[1,0].plot(test[3], label='Original')
axs[1,0].legend(loc="upper right")
axs[1,0].set(title="Señal 250 ruidosa (normalizada)")
axs[1,0].grid()
axs[1,0].set_xlabel('Muestras')
axs[1,0].set_ylabel('Magnitud')
plt.show() 

In [ ]:
#Concatenate fragments to build one song
cancion = test_noisy[10-1]
cancion_depurada = testDenoised[10-1]
for i in range(10, len(test)):
    cancion = np.concatenate((cancion, test_noisy[i]))
    cancion_depurada = np.concatenate((cancion_depurada, testDenoised[i]))
fig, ax = plt.subplots(figsize=(12,6), dpi=100)
plt.plot(cancion, label='Ruidosa')
plt.plot(cancion_depurada, label='Depurada')
plt.legend(loc="upper left")
plt.title('Fragmento de audio (test set)');
plt.grid()
plt.xlabel('Muestras')
plt.ylabel('Magnitud');

In [11]:
wavfile.write('testNoisy.wav', 44100, cancion)
wavfile.write('testDenoised.wav', 44100, cancion_depurada)

In [ ]:
#Real audio test
fs, audio_con_clicks = wavfile.read('audio_con_clicks.wav')
datos = []
for i in range(0, int(len(audio_con_clicks)/len(noisy[0]))):
    datos.append(audio_con_clicks[i*len(noisy[0]):(i + 1)*len(noisy[0])])

datos = np.array(datos)

datos = datos*0.05

m = 1000000
M = -1000000
for i in range(0, datos.shape[0]):
    M1 = max(datos[i])
    M = max(M1, M)
    
    m1 = min(datos[i])
    m = min(m1, m)  
for i in range(0, datos.shape[0]):
    datos[i] = (datos[i] - m)/(M - m)



In [ ]:
#Predict with the model
audioDenoised = model.predict(datos).reshape(datos.shape[0], datos.shape[1])

In [ ]:
#Concatenate
cancion = datos[0]
cancion_depurada = audioDenoised[0]
for i in range(1, len(audioDenoised)):
    cancion = np.concatenate((cancion, datos[i]))
    cancion_depurada = np.concatenate((cancion_depurada, audioDenoised[i]))
fig, ax = plt.subplots(figsize=(12,6), dpi=100)
plt.plot(cancion, label='Audio ruidoso real')
plt.plot(cancion_depurada, label='Audio real depurado')
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('Muestras')
plt.ylabel('Magnitud');

In [15]:
wavfile.write('audioDenoised.wav', 44100, cancion_depurada)

In [94]:
#Save trained model
model.save('working_v2.h5')

## Extra results

### Frequency domain

In [17]:
#To run this section, remember to generate the audio files from before
fs, test_noisy_spec = wavfile.read('testNoisy.wav')
fs, test_denoised_spec = wavfile.read('testDenoised.wav')
fs, audio_noisy_spec = wavfile.read('audio_con_clicks.wav')
fs, audio_denoised_spec = wavfile.read('audioDenoised.wav')

In [ ]:
from scipy.fft import fft, fftfreq

N = 10000
T = 1.0 / fs
x = np.linspace(0.0, N*T, N, endpoint=False)

yf = fft(test_noisy_spec)
xf = fftfreq(N, T)[:N//2]
yfd = fft(test_denoised_spec)

fig, ax = plt.subplots(figsize=(12,6), dpi=100)
plt.semilogy(xf[1:], 2.0/N * np.abs(yf[1:N//2]), label='Test ruidoso')
plt.semilogy(xf[1:], 2.0/N * np.abs(yfd[1:N//2]), label='Test depurado')
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Potencia [W]')
plt.legend(loc="upper right")
plt.grid()
plt.show()

In [ ]:
N = 10000
T = 1.0 / fs
x = np.linspace(0.0, N*T, N, endpoint=False)

yf = fft(audio_noisy_spec)
xf = fftfreq(N, T)[:N//2]
yfd = fft(audio_denoised_spec)


fig, ax = plt.subplots(figsize=(12,6), dpi=100)
plt.semilogy(xf[1:], 2.0/N * np.abs(yf[1:N//2]), label='Audio real ruidoso')
plt.semilogy(xf[1:], 2.0/N * np.abs(yfd[1:N//2]), label='Audio real depurado')
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Potencia [W]')
plt.legend(loc="upper left")
plt.grid()
plt.show()

### Number of clicks

In [ ]:
#Load data and predict with model
clean, noisy = dc.load_data_nueva(5000, 60, 20, 0.0034)
train, train_noisy, test, test_noisy, m, M = dc.normalizar([], [], clean, noisy)

test_denoised = model.predict(test_noisy).reshape(test.shape[0], test.shape[1])

In [22]:
#Denormalize
for i in range(0, len(test)):
    test_noisy[i] = (test_noisy[i])*(M - m) + m
    test[i] = (test[i])*(M - m) + m

In [ ]:

#Average number of clicks
cantidad = []
for i in range(0, len(test_noisy)):
    ubic = dc.deteccion(test_noisy[i], 0.9)
    cantidad.append(len(ubic))
clc = (sum(cantidad)/(len(test_noisy)*len(test_noisy[0])))
print("Ruidosos:")
print(clc)


cantidad = []
for i in range(0, len(test_denoised)):
    ubic = dc.deteccion(test_denoised[i], 0.9)
    cantidad.append(len(ubic))
clc = (sum(cantidad)/(len(test_denoised)*len(test_denoised[0])))
print("Limpios:")
print(clc)

In [24]:
#With real audio
fs, audio_con_clicks = wavfile.read('audio_con_clicks.wav')
datos = []
for i in range(0, int(len(audio_con_clicks)/len(noisy[0]))):
    datos.append(audio_con_clicks[i*len(noisy[0]):(i + 1)*len(noisy[0])])

datos = np.array(datos)

datos = datos*0.05

#Normalize

m = 1000000
M = -1000000
for i in range(0, datos.shape[0]):
    M1 = max(datos[i])
    M = max(M1, M)
    
    m1 = min(datos[i])
    m = min(m1, m)  
for i in range(0, datos.shape[0]):
    datos[i] = (datos[i] - m)/(M - m)


In [ ]:
#Predict with model and concatenate fragments
audioDenoised = model.predict(datos).reshape(datos.shape[0], datos.shape[1])
cancion = datos[0]
cancion_depurada = audioDenoised[0]
for i in range(1, len(audioDenoised)):
    cancion = np.concatenate((cancion, datos[i]))
    cancion_depurada = np.concatenate((cancion_depurada, audioDenoised[i]))

In [ ]:

#Number of clicks
ubic = dc.deteccion(cancion, 0.56)
clc = (len(ubic)/(len(cancion)))
print("Ruidoso:")
print(clc)

ubic = dc.deteccion(cancion_depurada, 0.56)
clc = (len(ubic)/(len(cancion_depurada)))
print("Limpio:")
print(clc)

### Mute audio

In [27]:
fs, clicks = wavfile.read('ruido.wav')
datos = []
for i in range(0, int(len(clicks)/149)):
    datos.append(clicks[i*149:(i + 1)*149])

datos = np.array(datos)

datos = datos*0.1

m = 1000000
M = -1000000
for i in range(0, datos.shape[0]):
    M1 = max(datos[i])
    M = max(M1, M)
    
    m1 = min(datos[i])
    m = min(m1, m)  
for i in range(0, datos.shape[0]):
    datos[i] = (datos[i] - m)/(M - m)


In [ ]:
silencio = model.predict(datos).reshape(datos.shape[0], datos.shape[1])

In [29]:
for i in range(0, silencio.shape[0]):
    silencio[i] = (silencio[i])*(M - m) + m
for i in range(0, datos.shape[0]):
    datos[i] = (datos[i])*(M - m) + m
datos.astype(int)
silencio.astype(int);

In [ ]:
cancion = datos[0]
cancion_depurada = silencio[0]
for i in range(1, len(silencio)):
    cancion = np.concatenate((cancion, datos[i]))
    cancion_depurada = np.concatenate((cancion_depurada, silencio[i]))
fig, ax = plt.subplots(figsize=(12,6), dpi=100)
plt.plot(cancion, label='Clicks solos')
plt.plot(cancion_depurada, label='Clicks luego de la red')
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('Muestras')
plt.ylabel('Magnitud');